
### Examining racial discrimination in the US job market

#### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

#### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes.

#### Exercise
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value.
   4. Discuss statistical significance.

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

****

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
# number of callbacks for black-sounding names
sum(data[data.race=='b'].call)

157.0

In [4]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1,0,1,0,0,0,0,0,0,
1,b,1,3,3,6,0,1,1,0,316,...,1,0,1,0,0,0,0,0,0,
2,b,1,4,1,6,0,0,0,0,19,...,1,0,1,0,0,0,0,0,0,
3,b,1,3,4,6,0,1,0,1,313,...,1,0,1,0,0,0,0,0,0,
4,b,1,3,3,22,0,0,0,0,313,...,1,1,0,0,0,0,0,1,0,Nonprofit


In [3]:
blacks = data[data.race=='b']
whites = data[data.race=='w']
bmean = sum(blacks.call)/len(blacks)
wmean = sum(whites.call)/len(whites)
allmean = sum(data.call)/len(data.call)
print('Black Mean: ',bmean)
print('White Mean: ',wmean)
print('Total Mean: ',allmean)

Black Mean:  0.064476386037
White Mean:  0.0965092402464
Total Mean:  0.0804928131417


Null Hypothesis: The black average is not significantly different from white or overall means.

Alternative: It is significantly less, p<0.05, which has a less than 1/20 chance under null.

Note: This works because black vs. white naming was assigned randomly to different resumes

In [4]:
#Why the is a series of 0 and 1 (booleans) stored as a float32, anyway?

calltotals = data.call.astype(int) 
callblacks = blacks.call.astype(int)
callwhites = whites.call.astype(int)
LB, LW = len(callblacks), len(callwhites)

startpool = callblacks, callwhites

#pool = np.concatenate((callblacks, callwhites.T), axis=0)
#hstack is equivalent to above w/ axis=1 and second item as a transpose (callwhites.T).
##(working with hstack and concatenate was a mess for a while :P)
#...I can't seem to get it to work, and I only need to concatenate for Rearrange. Whatever.

In [83]:
#Post-int-modification
print (type(callblacks))
print (callblacks.dtypes)
print (callblacks.unique())

<class 'pandas.core.series.Series'>
int64
[0 1]


In [95]:
group1, group2 = startpool
len(startpool)

2

In [6]:
group1, group2 = startpool
mixed = np.concatenate((group1, group2))
np.random.shuffle(mixed)
print (mixed)

[0 0 0 ..., 0 0 0]


In [28]:
#Get the relevant P-Value
def MeanDif(pool):
    group1, group2 = pool
    test_stat = group1.mean() - group2.mean()
    return test_stat

def Rearrange(pool):
    group1, group2 = pool
    mixed = np.concatenate((group1, group2))
    np.random.shuffle(mixed)
    return mixed[:LB], mixed[LB:]

def P_Val(pool, iters):
    real_stat = MeanDif(pool)
    null_stats = np.array([ MeanDif(Rearrange(pool)) for _ in range(iters) ])
    count = sum (real_stat <= null_stats)
    return count / iters

def CollectMeans(pool, iters):
    bucket=[]
    for _ in range(iters):
        bucket.append(MeanDif(Rearrange(pool)))
    return np.array(bucket)

#Yes, output it as an array. That lets numpy take std.


#Note to self: Do NOT assign a new variable the value of the shuffle while shuffling. Shuffles in place.
#Oh yay, I finally think I fixed everything

In [13]:
P_Val(startpool, 1000)

1.0

In [32]:
sample_means=CollectMeans(startpool, 1000)

In [33]:
sample_means[0:5]

array([-0.00574949,  0.00246407,  0.00328542,  0.00903491,  0.01067762])

In [34]:
margin_error = np.percentile(sample_means, [5, 95])
print (margin_error)

[-0.01314168  0.01314168]


In [35]:
confidence_interval = [allmean+margin_error[0], allmean+margin_error[1]]
print (confidence_interval)

[0.067351129363449683, 0.093634496919917862]


**Wow! p>0.999, that's highly significant!**

## Answers to problems

**1.What test is appropriate for this problem? Does CLT apply?**
   
This is a large sample, and while the relevant values (callbacks) are all either 0 or 1, the mean of many subsamples should still be expected to be a normally-distributed intermediate float somewhere between 0 and 1.
   
   
**2.What are the null and alternate hypotheses?**
   
Null Hypothesis: There is no significant difference betweeen callback rates for resumes randomly assigned black or white names.

Alternative Hypothesis: The rate of callbacks for resumes randomly assigned black names are significantly lower than those randomly assigned white names (p>0.95)

**3. Compute margin of error, confidence interval, and p-value.**

Margin of Error (around overall callbacks): 0.01314168

90% Confidence Interval (around overall callbacks): [ 0.067351129363449683, 0.093634496919917862 ]

p-value: >0.999

**4. Discuss statistical significance.**

Statistical significance indicates that the test produces results different enough from the results expected and modeled under the null hypothesis, that there's a less than 5% chance of them being generated if the null hypothesis is true. It lends some credence to alternative theories that might better explain the data observed.

In [37]:
#Other things
##stndrd_error_0 = sqrt( calltotals.mean()*(1-calltotals.mean) / len(calltotals) )
stndrd_error_1 = np.array(sample_means).std()
print(stndrd_error_1)

0.00808297620041


In [102]:
callblacks.mean()

0.06447638603696099

In [ ]:
std(calltotals)*